
# Storm Impact Prediction and Risk Analysis

## Objective
This project aims to analyze storm data to identify patterns, assess risk factors, and prepare the dataset for potential machine learning tasks such as storm severity classification or impact prediction.


In [4]:

# 1. Import Necessary Libraries
import pandas as pd
import numpy as np


In [5]:

# 2. Load the Dataset
df = pd.read_csv("storms.csv")
df.head()


,name,year,month,day,hour,lat,long,status,category,wind,pressure,tropicalstorm_force_diameter,hurricane_force_diameter
0,AL011975,1975,6,24,12,32.5,-52.0,tropical depression,NaN,20,-999,-1998,-1998
1,AL011975,1975,6,24,18,32.6,-52.6,tropical depression,NaN,25,-999,-1998,-1998
2,AL011975,1975,6,25,0,32.7,-53.2,tropical depression,NaN,25,-999,-1998,-1998
3,AL011975,1975,6,25,60,32.8,-53.2,tropical depression,NaN,25,-999,-1998,-1998
4,AL011975,1975,6,25,12,33.0,-54.5,tropical depression,NaN,25,-999,-1998,-1998


In [6]:

# 3. Explore the Dataset

# Basic Information
print("=== Dataset Information ===")
print(df.info())

# Statistical Summary
print("\n=== Dataset Description ===")
print(df.describe())

# Missing Values
print("\n=== Missing Values ===")
print(df.isnull().sum())


=== Dataset Information ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22705 entries, 0 to 22704
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   name                          22705 non-null  object 
 1   year                          22705 non-null  int64  
 2   month                         22705 non-null  int64  
 3   day                           22705 non-null  int64  
 4   hour                          22705 non-null  int64  
 5   lat                           22705 non-null  float64
 6   long                          22705 non-null  float64
 7   status                        22705 non-null  object 
 8   category                      5162 non-null   float64
 9   wind                          22705 non-null  int64  
 10  pressure                      22705 non-null  int64  
 11  tropicalstorm_force_diameter  22705 non-null  int64  
 12  hurricane_force_diameter      22